# Temporal Knowledge Graphs

## Overview

This notebook demonstrates how to build time-aware knowledge graphs, create snapshots, perform time-point queries, and track history.

### Learning Objectives

- Build temporal knowledge graphs with time information
- Create snapshots at specific time points
- Query the graph at specific times
- Track entity and relationship history over time

---

## Workflow

**Build Temporal KG → Create Snapshots → Time-Point Queries → Track History**

Temporal graphs enable time-aware queries and historical analysis.

---

## Step 1: Build Temporal Knowledge Graph

Build a knowledge graph with temporal support to track changes over time.


In [ ]:
from semantica.kg import GraphBuilder
import networkx as nx

entities = [
    {"id": "e1", "name": "Company X", "type": "Organization"},
    {"id": "e2", "name": "CEO A", "type": "Person"},
    {"id": "e3", "name": "CEO B", "type": "Person"},
]

relationships = [
    {"source": "e1", "target": "e2", "type": "has_ceo", "valid_from": "2020-01-01", "valid_to": "2023-12-31"},
    {"source": "e1", "target": "e3", "type": "has_ceo", "valid_from": "2024-01-01", "valid_to": None},
]

builder = GraphBuilder()

try:
    temporal_kg = nx.DiGraph()
    for entity in entities:
        temporal_kg.add_node(entity["id"], name=entity["name"], type=entity["type"])
    for rel in relationships:
        temporal_kg.add_edge(rel["source"], rel["target"], type=rel["type"], 
                            valid_from=rel.get("valid_from"), valid_to=rel.get("valid_to"))
    
    print(f"✓ Temporal knowledge graph built: {len(temporal_kg.nodes)} nodes, {len(temporal_kg.edges)} edges")
    print("  Note: Relationships have temporal validity periods")
    
except Exception as e:
    print(f"✗ Error building temporal graph: {e}")
    temporal_kg = None


## Step 2: Create Snapshots

Create snapshots of the graph at specific time points to capture the state at that moment.


In [ ]:
from semantica.kg import TemporalVersionManager

if temporal_kg is not None:
    version_manager = TemporalVersionManager()
    
    try:
        snapshot = version_manager.create_snapshot(temporal_kg, timestamp="2024-01-01")
        print("✓ Snapshot created for 2024-01-01")
        print(f"  Snapshot nodes: {len(snapshot.nodes) if snapshot else 'N/A'}")
        print("  Note: Snapshots capture graph state at specific time points")
        
    except Exception as e:
        print(f"✗ Error creating snapshot: {e}")
else:
    print("No temporal graph available for snapshots")


## Step 3: Time-Point Queries

Query the graph at specific time points to see what the graph looked like at that time.


In [ ]:
from semantica.kg import TemporalGraphQuery

if temporal_kg is not None:
    temporal_query = TemporalGraphQuery()
    
    try:
        graph_at_time = temporal_query.query_at_time(temporal_kg, "2024-01-01")
        print("✓ Queried graph at 2024-01-01")
        print(f"  Nodes at this time: {len(graph_at_time.nodes) if graph_at_time else 'N/A'}")
        
        changes = temporal_query.query_changes(temporal_kg, "2020-01-01", "2024-12-31")
        print(f"\n✓ Queried changes between 2020-01-01 and 2024-12-31")
        print(f"  Changes detected: {len(changes) if changes else 0}")
        
    except Exception as e:
        print(f"✗ Error querying temporal graph: {e}")
else:
    print("No temporal graph available for queries")


## Step 4: Track History

Track the evolution of specific entities and relationships over time.


In [ ]:
if temporal_kg is not None and 'temporal_query' in locals():
    try:
        entity_history = temporal_query.get_entity_history(temporal_kg, "e1")
        print("✓ Retrieved entity history")
        print(f"  History entries: {len(entity_history) if entity_history else 0}")
        
        if temporal_kg.edges():
            first_edge = list(temporal_kg.edges(data=True))[0]
            rel_id = f"{first_edge[0]}-{first_edge[1]}"
            relationship_history = temporal_query.get_relationship_history(temporal_kg, rel_id)
            print(f"\n✓ Retrieved relationship history")
            print(f"  History entries: {len(relationship_history) if relationship_history else 0}")
        
        print(f"\n✓ Temporal graph has {len(temporal_kg.nodes)} nodes across time")
        
    except Exception as e:
        print(f"✗ Error tracking history: {e}")
else:
    print("No temporal graph available for history tracking")
